In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import xgboost as xgb
from preprocess import preprocess_data
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [25]:
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
tqdm.pandas()

In [3]:
start = time.time()
df_train = pd.read_csv("raw_data/train_data.csv")
df_test = pd.read_csv("raw_data/test_data.csv")
end = time.time()
print("Data loaded in ", end - start, " seconds.")

Data loaded in  24.385851621627808  seconds.


In [4]:
df_train.head(1)

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,27,104.77,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [5]:
df_test.head(1)

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff
0,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,3180,3,4.5,1,2.94,0.0691,5.03,119.0,0,19222,1,10,2,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Preprocess Data

In [6]:
preprocessed_train = preprocess_data(df_train, split='train')
print('------')
preprocessed_test = preprocess_data(df_test, split='test')

------


In [7]:
preprocessed_train.head(1)

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,12,187,0.0,0.0,219,893,3,3.5,1,2.83,0.0438,4.95,27,104.77,0,23246,1,0,4,0,1,1,-330.0,1301.234406,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0


In [8]:
preprocessed_test.head(1)

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff
0,1,24,216,0.0,0.0,219,3180,3,4.5,1,2.94,0.0691,5.03,119.0,0,19222,1,10,2,0,1,0,-330.0,1312.801653,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
print(f'Size of Train data:{len(df_train)}')
print(f'Size of Test data:{len(df_test)}')

Size of Train data:4958347
Size of Test data:4959183


## Make Relevance Targets

Create a column called score which would be 
df['score'] = 4 * df.booking_bool + df.click_bool

In [10]:
preprocessed_train['rel'] = 4 * preprocessed_train.booking_bool + preprocessed_train.click_bool

In [11]:
X = preprocessed_train.drop(['booking_bool', 'click_bool', 'gross_bookings_usd', 'position'], axis=1)

In [12]:
print(f'num_features:{X.shape[1]-1}') # -1 as we donot use srch_id
X.head(1)

num_features:49


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,rel
0,1,12,187,0.0,0.0,219,893,3,3.5,1,2.83,0.0438,4.95,104.77,0,23246,1,0,4,0,1,1,-330.0,1301.234406,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


# Train Test Split

In [33]:
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(test_size=0.0001, n_splits=1, random_state = 7).split(X, groups=X['srch_id'])
X_train_inds, X_val_inds = next(gss)

In [34]:
assert len(X) == len(X_train_inds) + len(X_val_inds)

In [35]:
train_data= X.iloc[X_train_inds]
X_train = train_data.loc[:, ~train_data.columns.isin(['srch_id','rel'])]
y_train = train_data.loc[:, train_data.columns.isin(['rel'])]
train_groups = train_data.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

In [36]:
val_data= X.iloc[X_val_inds]
#We need to keep the id for later predictions
X_val = val_data.loc[:, ~val_data.columns.isin(['srch_id','rel'])]
X_val_with_srch_id = val_data.loc[:, ~val_data.columns.isin(['rel'])]
y_val = val_data.loc[:, val_data.columns.isin(['rel'])]
eval_groups = X_val_with_srch_id.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

In [88]:
len(X_val)

515

In [38]:
import xgboost as xgb

model = xgb.XGBRanker(  
    tree_method='gpu_hist',
    booster='gbtree',
    objective='rank:ndcg',
    random_state=42, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=100, 
    subsample=0.75,
    predictor='gpu_predictor'
    )

model = model.fit(X_train, y_train, group=train_groups, eval_set=[(X_val, y_val)], 
          eval_group = [eval_groups], eval_metric='ndcg@5', verbose=True)

[0]	validation_0-ndcg@5:0.29299
[1]	validation_0-ndcg@5:0.31525
[2]	validation_0-ndcg@5:0.29561
[3]	validation_0-ndcg@5:0.29783
[4]	validation_0-ndcg@5:0.27552
[5]	validation_0-ndcg@5:0.30127
[6]	validation_0-ndcg@5:0.30227
[7]	validation_0-ndcg@5:0.32161
[8]	validation_0-ndcg@5:0.33009
[9]	validation_0-ndcg@5:0.33009
[10]	validation_0-ndcg@5:0.30757
[11]	validation_0-ndcg@5:0.30757
[12]	validation_0-ndcg@5:0.30538
[13]	validation_0-ndcg@5:0.27987
[14]	validation_0-ndcg@5:0.28613
[15]	validation_0-ndcg@5:0.26679
[16]	validation_0-ndcg@5:0.26113
[17]	validation_0-ndcg@5:0.24179
[18]	validation_0-ndcg@5:0.26332
[19]	validation_0-ndcg@5:0.28178
[20]	validation_0-ndcg@5:0.28178
[21]	validation_0-ndcg@5:0.28178
[22]	validation_0-ndcg@5:0.27612
[23]	validation_0-ndcg@5:0.27612
[24]	validation_0-ndcg@5:0.27804
[25]	validation_0-ndcg@5:0.27612
[26]	validation_0-ndcg@5:0.28424
[27]	validation_0-ndcg@5:0.28350
[28]	validation_0-ndcg@5:0.28424
[29]	validation_0-ndcg@5:0.30246
[30]	validation_0-nd

In [39]:
model.feature_importances_.round(decimals=2)

array([0.01, 0.  , 0.01, 0.01, 0.01, 0.02, 0.05, 0.02, 0.01, 0.04, 0.35,
       0.02, 0.05, 0.22, 0.  , 0.01, 0.01, 0.01, 0.01, 0.01, 0.  , 0.  ,
       0.01, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.01,
       0.  , 0.  , 0.  , 0.01, 0.  , 0.01, 0.  , 0.  , 0.  , 0.  , 0.01,
       0.  , 0.01, 0.  , 0.01], dtype=float32)

In [79]:
def recommend_properties(model, data):
    result = pd.DataFrame(columns=['srch_id', 'prop_id', 'scores'])
    df_list = []
    grouped_data = data.groupby('srch_id')
    for group_name, group in tqdm(grouped_data):
        scores = model.predict(group.loc[:, ~group.columns.isin(['srch_id'])])
        sorted_group = sort_properties(group_name, scores, group['prop_id'])
        df_list.append(sorted_group)
    result = pd.concat(df_list)
    return result

In [66]:
def sort_properties(srch_id, scores, prop_ids):
    sorted_group = pd.DataFrame(columns=['srch_id', 'prop_id', 'scores'])
    sorted_group['srch_id'] = len(scores)*[srch_id]
    sorted_group['prop_id'] = prop_ids.values
    sorted_group['scores'] = scores
    sorted_group = sorted_group.sort_values(by='scores', ascending=False)
    return sorted_group 

In [90]:
start = time.time()
recommendations = recommend_properties(model, preprocessed_test)
end = time.time()
print(f'Total Time for validation prediction:{end - start}')
submission = recommendations.drop(columns=['scores'])
submission.to_csv('submission.csv', index=False)

100%|███████████████████████████████████████████████████████████| 199549/199549 [50:35<00:00, 65.73it/s]


Total Time for validation prediction:3065.540484905243


In [29]:
def recommend_properties_group(model, group):
    scores = model.predict(group.loc[:, ~group.columns.isin(['srch_id'])])
    sorted_group = sort_properties(group.name, scores, group['prop_id'])
    return sorted_group